# Data Preprocessing for Perfect Partner Probability Website Backend

This Jupyter Notebook is dedicated to the preprocessing of data from two surveys (`ASEC` and `NHANES`) for integration with the backend of the **[Perfect Partner Probability](https://perfectpartner.netlify.app/)**  website which utilizes Django Rest Framework.

In [1]:
# Import the neccesary packages
import pandas as pd 
import numpy as np
import sqlite3

## 2024 Annual Social and Economic Supplements (ASEC) by CPS (Current Population Survey)

### Data Dictionary ASEC

| Column Name | Description | Target Age Range | Values |
| --- | --- | --- | --- |
| A_AGE | Age of the participant | 0-79 years of age, 80-84 years of age, 85+ years of age | `00-79`: 0-79 years of age<br> `80`: 80-84 years of age<br> `85`: 85+ years of age |
| A_MARITL | Marital Status | All Persons | `1`: Married - civilian spouse present<br> `2`: Married - AF spouse present<br> `3`: Married - spouse absent (except separated)<br> `4`: Widowed<br> `5`: Divorced<br> `6`: Separated<br> `7`: Never married |
| A_SEX | Sex of the participant | All Persons | `1`: Male<br> `2`: Female |
| PRCITSHP | Citizenship Group | All Persons | `1`: Native, born in US<br> `2`: Native, born in PR or US outlying area<br> `3`: Native, born abroad of US parent(s)<br> `4`: Foreign born, US cit by naturalization<br> `5`: Foreign born, not a US citizen |
| PRDTRACE | Race | All Persons | `01`: White only<br> `02`: Black only<br> `03`: American Indian, Alaskan Native only (AI)<br> `04`: Asian only<br> `05`: Hawaiian/Pacific Islander only (HP)<br> `06`: White-Black<br> `07`: White-AI<br> `08`: White-Asian<br> `09`: White-HP<br> `10`: Black-AI<br> `11`: Black-Asian<br> `12`: Black-HP<br> `13`: AI-Asian<br> `14`: AI-HP<br> `15`: Asian-HP<br> `16`: White-Black-AI<br> `17`: White-Black-Asian<br> `18`: White-Black-HP<br> `19`: White-AI-Asian<br> `20`: White-AI-HP<br> `21`: White-Asian-HP<br> `22`: Black-AI-Asian<br> `23`: White-Black-AI-Asian<br> `24`: White-AI-Asian-HP<br> `25`: Other 3 race comb.<br> `26`: Other 4 or 5 race comb. |
| PTOTVAL | Total persons income | All Persons aged 15+ | `0`: none<br> Negative amount: Income (loss)<br> Positive amount: Income |


In [2]:
# Define the mapping of columns
column_to_keep = {
    "A_AGE": "age",
    "A_MARITL": "marital_status",
    "A_SEX": "sex",
    "PRCITSHP": "citizenship",
    "PRDTRACE": "race",
    "PTOTVAL": "income",
    "pwwgt0": "weights"
}

# Read the persons data
ASEC = pd.read_csv("pppub24.csv")

# Read the base statistical weights data for survey
base_weights = pd.read_csv("asec_csv_repwgt_2024.csv", usecols=["PPPOS", "h_seq", "pwwgt0"])

# Merge persons data with base weights using specified columns (Primary key / Foriegn Key)
ASEC = ASEC.merge(base_weights, left_on=["PPPOS", "PH_SEQ"], right_on=["PPPOS", "h_seq"], validate="1:1")

# Select and reorder columns based on the mapping for the final dataset
ASEC = ASEC[column_to_keep.keys()]

In [3]:
ASEC

,A_AGE,A_MARITL,A_SEX,PRCITSHP,PRDTRACE,PTOTVAL,pwwgt0
0,85,1,2,1,1,8565,601.574796
1,85,1,1,1,1,23641,601.574796
2,15,7,2,1,1,0,1312.044705
3,53,1,2,1,1,10801,1559.987507
4,52,1,1,1,1,40037,1559.987507
...,...,...,...,...,...,...,...
144260,46,1,1,1,1,80225,715.411338
144261,45,1,2,1,21,50601,715.411338
144262,14,7,2,1,21,0,600.527386
144263,10,7,2,1,21,0,489.614035


## 2021-2023 National Health and Nutrition Examination Survey (NHANES

### Data Dictionary (demographic fields need clarity only)

| Column Name     | Description                                      | Target Age Range | Values                                                                                      
|-----------------|--------------------------------------------------|-------------------|---------------------------------------------------------------------------------------------
| RIAGENDR        | Gender of the participant                        | 0 - 150 years     | `1`: Male<br> `2`: Female<br> `.`: Missing                                             
| RIDRETH3        | Race/Hispanic Origin with NH Asian                | 0 - 150 years     | `1`: Mexican American<br> `2`: Other Hispanic<br> `3`: Non-Hispanic White<br> `4`: Non-Hispanic Black<br> `6`: Non-Hispanic Asian<br> `7`: Other Race - Including Multi-Racial<br> `.`: Missing
| DMDMARTZ        | Marital Status                                   | 20 - 150 years    | `1`: Married/Living with Partner<br> `2`: Widowed/Divorced/Separated<br> `3`: Never married<br> `77`: Refused<br> `99`: Don't Know<br> `.`: Missing
| DMDBORN4      | Country of birth                                 | 0 - 150 years     | `1`: Born in 50 US states or Washington, DC<br> `2`: Others<br> `77`: Refused<br> `99`: Don't Know<br> `.`: Missing


In [47]:
# Define a dictionary for column renaming
measures_to_keep = {
    "RIAGENDR": "sex",
    "RIDRETH3": "race/HispanicOrigin w/ NH Asian",
    "DMDMARTZ": "maritalstatus",
    "RIDAGEYR": "age",
    "BMXWT": "weight_kg",
    "BMXHT": "height_cm",
    "BMXBMI": "bmi",
    "WTMEC2YR": "weights", # statistical weights for observations
    "DMDBORN4": "birth_place"
}

# Read and merge body measures with demographic data of NHANES for the final dataset
NHANES = (
    pd.read_sas("DEMO_L.XPT")
    .merge(pd.read_sas("BMX_L.XPT"), how="inner", on="SEQN", validate="1:1")
    [measures_to_keep.keys()]
)

In [48]:
NHANES

,RIAGENDR,RIDRETH3,DMDMARTZ,RIDAGEYR,BMXWT,BMXHT,BMXBMI,WTMEC2YR,DMDBORN4
0,1.0,6.0,1.0,43.0,86.9,179.5,27.0,54374.463898,2.0
1,1.0,3.0,1.0,66.0,101.8,174.2,33.5,34084.721548,1.0
2,2.0,2.0,1.0,44.0,69.4,152.9,29.7,81196.277992,2.0
3,2.0,7.0,NaN,5.0,34.3,120.1,23.8,55698.607106,1.0
4,1.0,3.0,NaN,2.0,13.6,NaN,NaN,36434.146346,1.0
...,...,...,...,...,...,...,...,...,...
8855,1.0,2.0,NaN,9.0,25.3,128.0,15.4,13459.129019,1.0
8856,2.0,4.0,3.0,49.0,NaN,143.8,NaN,64962.328962,1.0
8857,1.0,2.0,1.0,50.0,79.3,173.3,26.4,44367.534132,2.0
8858,1.0,2.0,2.0,40.0,81.9,179.1,25.5,46249.361849,1.0


---
* The ```harmonize_data``` function aligns **race, marital status, and born in the US** status across two dataframes (df1 and df2) using numeric codes for consistent analysis.

In [49]:
def harmonize_data(df1, df2):
    """
    Harmonizes race, marital status, and born in US status across two dataframes using numeric codes.

    Parameters:
    df1 (DataFrame): ASEC dataset.
    df2 (DataFrame): NHANES datset.
    """

    def harmonize_race(df1, df2):
        # Race mapping with numeric codes: 1 for White, 2 for Black, 3 for Asian, 4 for Other
        
        # Apply mapping to df1
        if 'PRDTRACE' in df1.columns:
            df1['race'] = df1['PRDTRACE'].map({1: 1, 2: 2, 4: 3}).fillna(4)  # Default to 4 (Other) for missing/unmapped values
        
        # Apply mapping to df2
        race_mapping_measurement = {3: 1, 4: 2, 6: 3, 1: 4, 2: 4, 7: 4}
        if 'RIDRETH3' in df2.columns:
            df2['race'] = df2['RIDRETH3'].map(race_mapping_measurement).fillna(4)  # Default to 4 (Other) for missing/unmapped values
        return df1, df2

    def harmonize_marital_status(df1, df2):
        # Marital status mapping: 1 for Married, 2 for Widowed/Divorced/Separated, 3 for Other
        
        # Apply mapping to df1
        df1['marital_status'] = df1['A_MARITL'].map({1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 2}).fillna(3)  # Default to 3 (Other) for missing/unmapped values
        # Apply mapping to df2
        df2['marital_status'] = df2['DMDMARTZ'].map({1: 1, 2: 2}).fillna(3)  # Default to 3 (Other) for missing/unmapped values
        return df1, df2

    def harmonize_Born_US(df1, df2):
        # Born in US status mapping: 1 for Yes, 0 for No
        # Apply mapping to df1
        df1['born_us'] = df1['PRCITSHP'].map({1: 1}).fillna(0)  # Default to 0 (No) for missing/unmapped values
        # Apply mapping to df2
        df2['born_us'] = df2['DMDBORN4'].map({1: 1}).fillna(0)  # Default to 0 (No) for missing/unmapped values
        return df1, df2

    # Apply each harmonization function in sequence
    df1, df2 = harmonize_race(df1, df2)
    df1, df2 = harmonize_marital_status(df1, df2)
    df1, df2 = harmonize_Born_US(df1, df2)

    # Drop the original columns
    df1 = df1.drop(["PRDTRACE", "A_MARITL", "PRCITSHP"], axis=1, errors='ignore')
    df2 = df2.drop(["RIDRETH3", "DMDMARTZ", "DMDBORN4"], axis=1, errors='ignore')

    return df1.rename(column_to_keep,axis=1), df2.rename(measures_to_keep,axis=1)

---
* Transfer the filtered data to sqlite to use it with the django backend for **Perfect Partner Probability** project

In [50]:
asec,nhanes = harmonize_data(ASEC,NHANES)

In [53]:
conn = sqlite3.connect('myDB.sqlite3')
asec.to_sql('Asec', conn, if_exists='replace')
nhanes.to_sql('Nhanes', conn, if_exists='replace')
asec.query("sex==1").to_sql("AsecMale",conn,if_exists='replace')
asec.query("sex==2").to_sql("AsecFemale",conn,if_exists='replace')
nhanes.query("sex==1").to_sql("NhanesMale",conn,if_exists='replace')
nhanes.query("sex==2").to_sql("NhanesFemale",conn,if_exists='replace')
conn.close()

In [7]:
ASEC['PTOTVAL']

0          8565
1         23641
2             0
3         10801
4         40037
          ...  
144260    80225
144261    50601
144262        0
144263        0
144264        0
Name: PTOTVAL, Length: 144265, dtype: int64

In [13]:
# Define the bins (thresholds)
thresholds = np.arange(0, 500001, 5000)  # from 0 to 500000 by 5000

# Calculate quantiles for each threshold
quantiles = [(ASEC.query("A_SEX==1")['PTOTVAL'] <= t).mean() for t in thresholds]

# Put in DataFrame for display
quantile_df = pd.DataFrame({
    'Income Threshold': thresholds,
    'Quantile (Percentile)': [round(q * 100, 2) for q in quantiles]
})

In [17]:
ASEC['PTOTVAL'].min()

-20043

### find income percentile for line plot

In [62]:
t = np.arange(0, 500001, 5000)
w = ASEC.query("A_AGE>=15 & A_SEX==1")['pwwgt0'].to_numpy()
x = ASEC.query("A_AGE>=15 & A_SEX==1")['PTOTVAL'].to_numpy()

percentiles = [(w[x <= i].sum() / w.sum()) for i in t]
pd.DataFrame({'income': t, 'income_percentile': np.round(percentiles, 4)}).to_json('income_percentiles_male.json', orient='records', indent=1)

w = ASEC.query("A_AGE>=15 & A_SEX==2")['pwwgt0'].to_numpy()
x = ASEC.query("A_AGE>=15 & A_SEX==2")['PTOTVAL'].to_numpy()

percentiles = [(w[x <= i].sum() / w.sum())  for i in t]
pd.DataFrame({'income': t, 'income_percentile': np.round(percentiles, 4)}).to_json('income_percentiles_female.json', orient='records', indent=1)

### find income bins for histogram

In [79]:
bins = np.arange(0, 500001, 10000)
df = ASEC.query("A_AGE>=15 & A_SEX==1")
income_bins_male = [
    {
        'population': df.loc[(df['PTOTVAL'] >= left) & (df['PTOTVAL'] < right), 'pwwgt0'].sum(),
        'left': left,
        'right': right
    }
    for left, right in zip(bins[:-1], bins[1:])
]
pd.DataFrame(income_bins_male).to_json('income_bins_male.json', orient='records', indent=1)
# -------------------

df = ASEC.query("A_AGE>=15 & A_SEX==2")
income_bins_male = [
    {
        'population': df.loc[(df['PTOTVAL'] >= left) & (df['PTOTVAL'] < right), 'pwwgt0'].sum(),
        'left': left,
        'right': right
    }
    for left, right in zip(bins[:-1], bins[1:])
]
pd.DataFrame(income_bins_male).to_json('income_bins_female.json', orient='records', indent=1)

In [80]:
bins = np.arange(0, 500001, 5000)
df = ASEC.query("A_AGE>=15 & A_SEX==1")
income_bins_male = [
    {
        'population': df.loc[(df['PTOTVAL'] >= left) & (df['PTOTVAL'] < right), 'pwwgt0'].sum(),
        'left': left,
        'right': right
    }
    for left, right in zip(bins[:-1], bins[1:])
]
pd.DataFrame(income_bins_male).to_json('income_bins_male_5k.json', orient='records', indent=1)
# -------------------

df = ASEC.query("A_AGE>=15 & A_SEX==2")
income_bins_male = [
    {
        'population': df.loc[(df['PTOTVAL'] >= left) & (df['PTOTVAL'] < right), 'pwwgt0'].sum(),
        'left': left,
        'right': right
    }
    for left, right in zip(bins[:-1], bins[1:])
]
pd.DataFrame(income_bins_male).to_json('income_bins_female_5k.json', orient='records', indent=1)

In [81]:
bins = np.arange(0, 500001, 20000)
df = ASEC.query("A_AGE>=15 & A_SEX==1")
income_bins_male = [
    {
        'population': df.loc[(df['PTOTVAL'] >= left) & (df['PTOTVAL'] < right), 'pwwgt0'].sum(),
        'left': left,
        'right': right
    }
    for left, right in zip(bins[:-1], bins[1:])
]
pd.DataFrame(income_bins_male).to_json('income_bins_male_20k.json', orient='records', indent=1)
# -------------------

df = ASEC.query("A_AGE>=15 & A_SEX==2")
income_bins_male = [
    {
        'population': df.loc[(df['PTOTVAL'] >= left) & (df['PTOTVAL'] < right), 'pwwgt0'].sum(),
        'left': left,
        'right': right
    }
    for left, right in zip(bins[:-1], bins[1:])
]
pd.DataFrame(income_bins_male).to_json('income_bins_female_20k.json', orient='records', indent=1)

In [85]:
over500kIncomeMale = ASEC.query("A_AGE>=15 & A_SEX==1 & PTOTVAL>500000")['pwwgt0'].sum()
over500kIncomeFemale = ASEC.query("A_AGE>=15 & A_SEX==2 & PTOTVAL>500000")['pwwgt0'].sum()

In [86]:
over500kIncomeMale,over500kIncomeFemale

(1059096.0728130098, 397306.76846411)